In [1]:
import ROOT as root
import numpy as np
import random
from timer import Timer

t = Timer()
t.start()

Welcome to JupyROOT 6.24/08


In [2]:
fileName = "rootfiles/test42.root"
treeName = "tdata"

f = root.TFile(fileName)
myTree = f.Get(treeName)

myTree.Print()

root.EnableImplicitMT()
d = root.RDataFrame(treeName,fileName)

******************************************************************************
*Tree    :tdata     : Run 42                                                 *
*Entries :     5770 : Total =          741702 bytes  File  Size =     151563 *
*        :          : Tree compression factor =   4.90                       *
******************************************************************************
*Br    0 :adc       : adc[16]/I                                              *
*Entries :     5770 : Total  Size=     370677 bytes  File Size  =     105636 *
*Baskets :       12 : Basket Size=      32000 bytes  Compression=   3.50     *
*............................................................................*
*Br    1 :tdc       : tdc[16]/I                                              *
*Entries :     5770 : Total  Size=     370677 bytes  File Size  =      45298 *
*Baskets :       12 : Basket Size=      32000 bytes  Compression=   8.17     *
*...................................................

Warning in <EnableImplicitMT>: Cannot enable implicit multi-threading with 0 threads, please build ROOT with -Dimt=ON


In [3]:
triggercode='''
using namespace std;
using RNode = ROOT::RDF::RNode;
std::vector<RNode> v;

bool remakepedfile= false;
const int bin = 100;
const Int_t Nadc = 16;
const Int_t Ntdc = 16;
const Int_t pedrun = 172;
const Int_t bl=Ntdc-16+0;//Start at channel 0
const Int_t br=Ntdc-16+1;
const Int_t tl=Ntdc-16+2;
const Int_t tr=Ntdc-16+3;
const Int_t tdc_min = 500;
const Int_t tdc_max = 3500;
const Double_t adjadcto=1400.0;//value to ADJust ADC TO
Double_t t_fullscale = 140.0E-09; // full scale TDC range in seconds
Double_t t_convert=t_fullscale/4096.0;

const Int_t nthetabins = 51;
const Double_t thetalow = -100.5;
const Double_t thetahigh = 100.5;

const Double_t nscint=1.50;

Double_t vn = 2.997E08/nscint;
Double_t resolution = 0.934*(0.0232*nscint*nscint-0.1061*nscint+0.1617);
Double_t granularity = t_convert*vn/2.0;
Double_t xpos_range = 0.30;
Double_t dscint = 0.105; // distance between scintillators in metres
const int xposbin = 2.0*xpos_range/granularity;

Char_t tdcnames[][Ntdc]={"Bottom Left","Bottom Right","Top Left","Top Right","4","5","6","7","8","9","10","11","12","13","14","15"};
Char_t adcnames[][Nadc]={"Bottom Left","Bottom Right","Top Left","Top Right","4","5","6","7","8","9","10","11","12","13","14","15"};

Double_t tdccorrect[Ntdc];
Double_t ped[Nadc];
Double_t gain[Nadc];

TRandom r;
Double_t rnd;

bool getTrigger(int* tdc, int* adc) {

        bool tdc_tl = false;
        bool tdc_tr = false;
        bool tdc_bl = false;
        bool tdc_br = false;
        bool trigger = false;

        if (tdc[tl]>tdc_min&&tdc[tl]<tdc_max) {
                tdc_tl = true;
        }
        if (tdc[tr]>tdc_min&&tdc[tr]<tdc_max) {
                tdc_tr = true;
        }
        if (tdc[bl]>tdc_min&&tdc[bl]<tdc_max) {
                tdc_bl = true;
        }
        if (tdc[br]>tdc_min&&tdc[br]<tdc_max) {
                tdc_br = true;
        }

        if (tdc_tl && tdc_tr && tdc_bl && tdc_br) {
                trigger = true;
        }

        return trigger;
}

TCanvas* calculateADCFactors() {

        ROOT::RDF::RResultPtr<TH1D> hadcraw[Ntdc];
        TCanvas *c3 = new TCanvas("c3", "c3", 100,100,600,600);
        c3->Divide(2,2, 0.01, 0.01, 0);

        //=====================================GET PED for ADC==========================
        if(remakepedfile){cout << "Executing luterpedestals.C" << endl; 
        gROOT->ProcessLine(Form(".x luterpedestals.C(%d)",pedrun));
        cout<<"pedestal file made"<<endl;}

        //cout << "Opening pedestal values file ... " << endl;
        FILE *adcpeds = fopen(Form("./pedestalfiles/pedestalrun%d.dat",pedrun),"r");

        //cout << "Filling ADC pedestal array ..." << endl;
        for( int i = 0; i < Nadc ; i++) {
                  fscanf(adcpeds,"%lf",&ped[i]);
        }

        for (int i = bl; i <= tr ; i++) {
                auto varname = "adc[" + std::to_string(i) + "]";
                hadcraw[i] = v[0].Define(Form("hadc%02d",i),varname).Histo1D({Form("hadc%02d",i), adcnames[i], bin, 0, 3000.0},Form("hadc%02d",i));
                gain[i] = adjadcto/(hadcraw[i]->GetMean()-ped[i]);
                cout << i << " " << hadcraw[i]->GetMean() << " " << ped[i] << " " << gain[i] << endl;
                c3->cd(i+1);
                hadcraw[i]->DrawClone();
        }

        return c3;

}

TCanvas* calculateTOffsets(){

        ROOT::RDF::RResultPtr<TH1D> htdcraw[Ntdc];
        TCanvas *c1 = new TCanvas("c1", "c1", 75,75,600,600);
        c1->Divide(2,2, 0.01, 0.01, 0);

        for (int i = bl; i <= tr ; i++) {
                auto varname = "tdc[" + std::to_string(i) + "]";
                htdcraw[i] = v[0].Define(Form("htdc%02d",i),varname).Histo1D({Form("htdc%02d",i), tdcnames[i], bin, 1700, 2100.0},Form("htdc%02d",i));
                cout << i << " " << htdcraw[i]->GetMean() << " " << htdcraw[i]->GetStdDev() << endl;
                c1->cd(i+1);
                int highbin=5;
                for (int ii=0;ii<=bin;ii++) {
                        if ((htdcraw[i]->GetBinContent(ii))>(htdcraw[i]->GetBinContent(highbin))&&ii>=5){
                                highbin=ii;
                        }
                }
                int median = htdcraw[i]->GetBinCenter(highbin);
                htdcraw[i]->Fit("gaus","Q","",(median-200),(median+200));
                tdccorrect[i] = 2000 - (htdcraw[i]->GetFunction("gaus")->GetParameter(1));
                htdcraw[i]->DrawClone();
        }

        return c1;

}

float getTheta2(bool trigger, float xtop, float xbottom) {

        double rtod=180.0/3.14159265;
        Double_t rnd_top_pos = r.Gaus(0.0,resolution);
        Double_t rnd_bottom_pos = r.Gaus(0.0,resolution);

        TF1 *fphi = new TF1("fsin", "x", 0, 2*TMath::Pi());
        TF1 *fcos = new TF1("fcos", "cos(x)*cos(x)",-TMath::Pi()/2.0,TMath::Pi()/2.0);
        Double_t phisim, cossim, rndxt, rndyt, rndrt, rndrr, rndxb, rndyb;
        while (true) {
            phisim = fphi->GetRandom();
            cossim = fcos->GetRandom();
            rndxt = r.Uniform(-0.10,0.10);
            rndyt = r.Uniform(-0.15,0.15);
            rndrt = sqrt(rndxt*rndxt+rndyt*rndyt)+rnd_top_pos;
            rndrr = dscint*tan(cossim);
            rndxb = rndxt+rndrr*cos(phisim);
            rndyb = rndyt+rndrr*sin(phisim);
            if (rndxb > -0.10 && rndxb < 0.10 && rndyb > -0.15 && rndyb < 0.15) break;
        }

        Double_t rndrb = sqrt(rndxb*rndxb+rndyb*rndyb)+rnd_bottom_pos;
        Double_t theta2 = 0.8281-rtod*atan((rndrt-rndrb)/dscint);

        if (trigger) {
                return theta2;
        }

        return -1000;

}

float getAdcTL(bool trigger, int* adc) {

        //std::vector<float> v;

        if(trigger) {
                //v.push_back(gain[tl]*(adc[tl]-ped[tl]));
                return gain[tl]*(adc[tl]-ped[tl]);
        }

        return -1;
}

float getAdcTR(bool trigger, int* adc) {

        //std::vector<float> v;

        if(trigger) {
                return gain[tr]*(adc[tr]-ped[tr]);
        }

        return -1;
}

float getAdcBL(bool trigger, int* adc) {

        //std::vector<float> v;

        if(trigger) {
                return gain[bl]*(adc[bl]-ped[bl]);
        }

        return -1;
}

float getAdcBR(bool trigger, int* adc) {

        //std::vector<float> v;

        if(trigger) {
                return gain[br]*(adc[br]-ped[br]);
        }

        return -1;
}

float getTdcTL(bool trigger, int* tdc) {

        //std::vector<float> v;

        if(trigger) {
                //v.push_back(tdc[tl]+tdccorrect[tl]);
                return tdc[tl]+tdccorrect[tl];
        }

        return -1;
}

float getTdcTR(bool trigger, int* tdc) {

        //std::vector<float> v;

        if(trigger) {
                //v.push_back(tdc[tr]+tdccorrect[tr]);
                return tdc[tr]+tdccorrect[tr];
        }

        return -1;
}

float getTdcBL(bool trigger, int* tdc) {

        //std::vector<float> v;

        if(trigger) {
                //v.push_back(tdc[bl]+tdccorrect[bl]);
                return tdc[bl]+tdccorrect[bl];
        }

        return -1;
}

float getTdcBR(bool trigger, int* tdc) {

        //std::vector<float> v;

        if(trigger) {
                //v.push_back(tdc[br]+tdccorrect[br]);
                return tdc[br]+tdccorrect[br];
        }

        return -1;
}

float getXTop(bool trigger, double ttl, double ttr, double tbl, double tbr) {

        //std::vector<float> v;

        if(trigger) {
                return (ttl-ttr)/2.0*t_convert*vn;
        }

        return -1000;
}

float getXMeanTop(bool trigger, double ttl, double ttr, double tbl, double tbr) {

        //std::vector<float> v;

        if(trigger) {
                return (ttl+ttr-4000.0)*t_convert*vn;
        }

        return -1000;
}

float getXMean(bool trigger, double xmeantop, double xmeanbottom) {

        //std::vector<float> v;

        if(trigger) {
                return (xmeantop-xmeanbottom)/2.0;
        }

        return -1000;
}

float getXBottom(bool trigger, double ttl, double ttr, double tbl, double tbr) {

        //std::vector<float> v;

        if(trigger) {
                return (tbl-tbr)/2.0*t_convert*vn;
        }

        return -1000;
}

float getXMeanBottom(bool trigger, double ttl, double ttr, double tbl, double tbr) {

        //std::vector<float> v;

        if(trigger) {
                return (tbl+tbr-4000.0)*t_convert*vn;
        }

        return -1000;
}

float getERatio(bool trigger, double etop, double ebottom) {

        if (trigger) {
                return etop/ebottom;
        }

        return -1000;
}

float getETop(bool trigger, double adctl, double adctr) {

        if (trigger) {
                return (adctr+adctl)/2.0;
        }

        return -1000;
}

float getEBottom(bool trigger, double adcbl, double adcbr) {

        if (trigger) {
                return (adcbr+adcbl)/2.0;
        }

        return -1000;
}
'''

root.gInterpreter.Declare(triggercode)

True

In [5]:
fdf = d.Define("trigger","getTrigger(&tdc[0],&adc[0])")

entries = d.Count();
print('{} entries'.format(entries.GetValue()))

triggers = fdf.Filter("trigger==true").Count()
print('{} entries passed trigger'.format(triggers.GetValue()))

fdft = fdf.Filter("trigger==true")

5770 entries
3340 entries passed trigger
